In [1]:
import time
import warnings
import pandas as pd
import numpy as np
import os
import psutil
from sklearn.base import clone
from sklearn.datasets import make_classification
from memory_profiler import memory_usage

# --- Your other imports ---
from skrebate import ReliefF, SURF, SURFstar, MultiSURF as SkrebateMultiSURF, MultiSURFstar
from src.fast_select.ReliefF import ReliefF as FastReliefF
from src.fast_select.SURF import SURF as FastSURF
from src.fast_select.MultiSURF import MultiSURF as FastMultiSURF

# --- GPU Detection ---
try:
    from numba import cuda
    GPU_AVAILABLE = cuda.is_available()
except (ImportError, cuda.cudadrv.error.CudaSupportError):
    GPU_AVAILABLE = False

# --- Benchmark Configuration ---
P_DOMINANT_SCENARIOS = {
    "name": "p >> n (Features Dominant)",
    "fixed_param": "n_samples", "fixed_value": 500,
    "varied_param": "n_features", "varied_range": [100, 200, 300, 400, 500]
}
N_DOMINANT_SCENARIOS = {
    "name": "n >> p (Samples Dominant)",
    "fixed_param": "n_features", "fixed_value": 100,
    "varied_param": "n_samples", "varied_range": [500, 1000, 1500, 2000, 2500]
}
N_FEATURES_TO_SELECT = 10
N_REPEATS = 3 # Increase repeats for more stable results

# --- Estimators to Test ---
estimators = {
    # skrebate estimators
    "skrebate.ReliefF": ReliefF(n_features_to_select=N_FEATURES_TO_SELECT, n_neighbors=10, n_jobs=-1),
    "skrebate.SURF": SURF(n_features_to_select=N_FEATURES_TO_SELECT, n_jobs=-1),
    "skrebate.MultiSURF": SkrebateMultiSURF(n_features_to_select=N_FEATURES_TO_SELECT, n_jobs=-1),
    # fast-select CPU estimators
    "fast_select.ReliefF (CPU)": FastReliefF(n_features_to_select=N_FEATURES_TO_SELECT, n_neighbors=10, backend='cpu', n_jobs=-1),
    "fast_select.SURF (CPU)": FastSURF(n_features_to_select=N_FEATURES_TO_SELECT, n_jobs=-1),
    "fast_select.MultiSURF (CPU)": FastMultiSURF(n_features_to_select=N_FEATURES_TO_SELECT, backend='cpu', n_jobs=-1),
}
if GPU_AVAILABLE:
    print("NVIDIA GPU detected. Including GPU benchmarks.")
    estimators.update({
        "fast_select.ReliefF (GPU)": FastReliefF(n_features_to_select=N_FEATURES_TO_SELECT, n_neighbors=10, backend='gpu'),
        "fast_select.SURF (GPU)": FastSURF(n_features_to_select=N_FEATURES_TO_SELECT, backend='gpu'),
        "fast_select.MultiSURF (GPU)": FastMultiSURF(n_features_to_select=N_FEATURES_TO_SELECT, backend='gpu'),
    })
else:
    print("No NVIDIA GPU detected. Skipping GPU benchmarks.")

# --- CORRECTED BENCHMARK FUNCTION ---
def run_single_benchmark(estimator, X, y):
    """
    Measures runtime and peak memory usage of a single estimator fit.
    This version performs only ONE execution and correctly measures GPU memory.
    """
    is_gpu_estimator = hasattr(estimator, 'backend') and estimator.backend == 'gpu'
    
    # Use a lambda to wrap the fit call
    fit_func = lambda: estimator.fit(X, y)

    # --- Memory Measurement ---
    peak_mem_mb = 0
    if is_gpu_estimator:
        # For GPU, we measure VRAM usage directly.
        # This requires the fit function to be run inside the context.
        ctx = cuda.current_context()
        start_mem = ctx.get_memory_info().free
        fit_func() # Run the function
        end_mem = ctx.get_memory_info().free
        # Peak memory is the reduction in free memory.
        peak_mem_mb = (start_mem - end_mem) / (1024**2)
    else:
        # For CPU, memory_profiler works perfectly.
        mem_samples = memory_usage(fit_func, interval=0.1)
        peak_mem_mb = max(mem_samples)

    # --- Runtime Measurement ---
    # Since the function has already been run once for memory profiling,
    # we time a second run to get a pure execution time without JIT overhead.
    # This is now a consistent measurement.
    start_time = time.perf_counter()
    fit_func()
    end_time = time.perf_counter()
    runtime = end_time - start_time
    
    return runtime, peak_mem_mb

def warmup_jit_compilers(estimators_dict):
    """Performs a 'warm-up' run on a small dataset to compile JIT functions."""
    print("\n--- Warming up JIT compilers ---")
    X_warmup, y_warmup = make_classification(n_samples=20, n_features=20, random_state=42)
    for name, estimator in estimators_dict.items():
        # More robust check for our custom estimators
        if "fast_select" in name:
            print(f"  Warming up {name}...")
            try:
                # Use a fresh clone for warmup
                clone(estimator).fit(X_warmup, y_warmup)
            except Exception as e:
                warnings.warn(f"  > Warm-up FAILED for {name}. Reason: {type(e).__name__}: {e}")
    print("--- Warm-up complete ---")

def main():
    """Main function to run all benchmark scenarios."""
    results = []
    warmup_jit_compilers(estimators)

    scenarios = [P_DOMINANT_SCENARIOS, N_DOMINANT_SCENARIOS]
    
    for scenario_params in scenarios:
        scenario_name = scenario_params["name"]
        print(f"\n--- Running Scenario: {scenario_name} ---")
        
        fixed_param = scenario_params["fixed_param"]
        varied_param = scenario_params["varied_param"]
        
        for varied_value in scenario_params["varied_range"]:
            # Set up dataset dimensions for this run
            if fixed_param == "n_samples":
                n_samples = scenario_params["fixed_value"]
                n_features = varied_value
            else:
                n_samples = varied_value
                n_features = scenario_params["fixed_value"]
                
            print(f"\nGenerating data: {n_samples} samples, {n_features} features")
            X, y = make_classification(n_samples=n_samples, n_features=n_features, n_informative=20, n_redundant=50, random_state=42)

            for name, base_estimator in estimators.items():
                for i in range(N_REPEATS):
                    print(f"  Benchmarking {name} (Run {i+1}/{N_REPEATS})...")
                    try:
                        estimator = clone(base_estimator)
                        runtime, peak_mem = run_single_benchmark(estimator, X, y)
                        results.append({
                            "scenario": scenario_name, "algorithm": name,
                            "n_samples": n_samples, "n_features": n_features,
                            "runtime_sec": runtime, "peak_memory_mb": peak_mem
                        })
                    except Exception as e:
                        warnings.warn(f"  > FAILED: {name} on {n_samples}x{n_features}. Reason: {type(e).__name__}: {e}", UserWarning)

    # --- Save results to CSV ---
    df = pd.DataFrame(results)
    output_file = "benchmark_results_with_memory.csv"
    df.to_csv(output_file, index=False)
    print(f"\nBenchmarking complete. Results saved to '{output_file}'")

if __name__ == "__main__":
    main()

NVIDIA GPU detected. Including GPU benchmarks.

--- Warming up JIT compilers ---
  Warming up fast_select.ReliefF (CPU)...


/home/galynch/snap/snapd-desktop-integration/253/Desktop/FastSelect/src/fast_select/ReliefF.py:360: UserWarning: n_neighbors (10) is greater than or equal to the smallest class size (10).
  warnings.warn(
/home/galynch/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


  Warming up fast_select.SURF (CPU)...
  Warming up fast_select.MultiSURF (CPU)...
  Warming up fast_select.ReliefF (GPU)...


/home/galynch/snap/snapd-desktop-integration/253/Desktop/FastSelect/src/fast_select/ReliefF.py:360: UserWarning: n_neighbors (10) is greater than or equal to the smallest class size (10).
  warnings.warn(


  Warming up fast_select.SURF (GPU)...
  Warming up fast_select.MultiSURF (GPU)...
--- Warm-up complete ---

--- Running Scenario: p >> n (Features Dominant) ---

Generating data: 500 samples, 100 features
  Benchmarking skrebate.ReliefF (Run 1/3)...
  Benchmarking skrebate.ReliefF (Run 2/3)...
  Benchmarking skrebate.ReliefF (Run 3/3)...
  Benchmarking skrebate.SURF (Run 1/3)...
  Benchmarking skrebate.SURF (Run 2/3)...
  Benchmarking skrebate.SURF (Run 3/3)...
  Benchmarking skrebate.MultiSURF (Run 1/3)...
  Benchmarking skrebate.MultiSURF (Run 2/3)...
  Benchmarking skrebate.MultiSURF (Run 3/3)...
  Benchmarking fast_select.ReliefF (CPU) (Run 1/3)...
  Benchmarking fast_select.ReliefF (CPU) (Run 2/3)...
  Benchmarking fast_select.ReliefF (CPU) (Run 3/3)...
  Benchmarking fast_select.SURF (CPU) (Run 1/3)...
  Benchmarking fast_select.SURF (CPU) (Run 2/3)...
  Benchmarking fast_select.SURF (CPU) (Run 3/3)...
  Benchmarking fast_select.MultiSURF (CPU) (Run 1/3)...
  Benchmarking fast_s

  Benchmarking fast_select.ReliefF (GPU) (Run 1/3)...
  Benchmarking fast_select.ReliefF (GPU) (Run 2/3)...
  Benchmarking fast_select.ReliefF (GPU) (Run 3/3)...
  Benchmarking fast_select.SURF (GPU) (Run 1/3)...
  Benchmarking fast_select.SURF (GPU) (Run 2/3)...
  Benchmarking fast_select.SURF (GPU) (Run 3/3)...
  Benchmarking fast_select.MultiSURF (GPU) (Run 1/3)...
  Benchmarking fast_select.MultiSURF (GPU) (Run 2/3)...
  Benchmarking fast_select.MultiSURF (GPU) (Run 3/3)...

Generating data: 1000 samples, 100 features
  Benchmarking skrebate.ReliefF (Run 1/3)...
  Benchmarking skrebate.ReliefF (Run 2/3)...
  Benchmarking skrebate.ReliefF (Run 3/3)...
  Benchmarking skrebate.SURF (Run 1/3)...
  Benchmarking skrebate.SURF (Run 2/3)...
  Benchmarking skrebate.SURF (Run 3/3)...
  Benchmarking skrebate.MultiSURF (Run 1/3)...
  Benchmarking skrebate.MultiSURF (Run 2/3)...
  Benchmarking skrebate.MultiSURF (Run 3/3)...
  Benchmarking fast_select.ReliefF (CPU) (Run 1/3)...
  Benchmarking f

In [2]:
!git commit -a -m "documentation updates: added user guide"
!git push

[development 58cfac2] documentation updates: added user guide
Enumerating objects: 25, done.
Counting objects: 100% (19/19), done.
Delta compression using up to 20 threads
Compressing objects: 100% (11/11), done.
Writing objects: 100% (11/11), 5.33 KiB | 2.66 MiB/s, done.
Total 11 (delta 7), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (7/7), completed with 6 local objects.
To https://github.com/GavinLynch04/FastSelect.git
   fe234e0..58cfac2  development -> development


In [3]:
!git pull


Already up to date.


In [7]:
!git add ../docs/source/user_guide.md

v12.22.9


In [2]:
!python3 -m pytest

.................sFFFFFFF...........s..........................s........ [ 88%]
.........                                                                [100%]
=================================== FAILURES ===================================
_________________________ test_fit_transform_cpu[MID] __________________________

discrete_classification_data = (array([[3, 0, 1, ..., 3, 1, 2],
       [0, 1, 3, ..., 0, 2, 3],
       [3, 1, 0, ..., 3, 1, 0],
       ...,
       [0...,
       2, 0, 1, 3, 2, 1, 0, 2, 3, 0, 0, 2, 1, 2, 3, 3, 1, 3, 1, 2, 0, 1,
       2, 0, 0, 2, 0, 2, 3, 2, 3, 2, 0, 0]))
method = 'MID'

    @pytest.mark.parametrize("method", ['MID', 'MIQ'])
    def test_fit_transform_cpu(discrete_classification_data, method):
        """
        Test the full fit and transform cycle on the CPU backend for both methods.
        This covers the CPU host caller and JIT kernels.
        """
        X, y = discrete_classification_data
        n_samples, n_features = X.shape
        n_select

In [4]:
!find . -type d -name ".pytest_cache" -exec rm -r {} +



In [14]:
from skrebate import MultiSURFstar as SKMultiSURF
from skrebate import SURFstar as SURFSK
from skrebate import ReliefF as ReliefFSK
import numpy as np
def generate_large_robust_dataset(n_samples=1000, seed=29):
    np.random.seed(seed)
    # Create balanced binary classes
    n0 = n_samples // 2
    n1 = n_samples - n0
    y = np.array([0] * n0 + [1] * n1, dtype=np.int32)

    # Feature 0: Highly relevant continuous feature.
    # Class 0 samples come from N(1, 1), class 1 samples come from N(10, 1).
    f0_class0 = np.random.normal(loc=1.0, scale=1.0, size=n0)
    f0_class1 = np.random.normal(loc=10.0, scale=1.0, size=n1)
    f0 = np.concatenate([f0_class0, f0_class1])

    # Feature 1: Irrelevant noise feature.
    # Values drawn from a standard normal distribution regardless of class.
    f1 = np.random.normal(loc=0.0, scale=1.0, size=n_samples)

    # Feature 2: Relevant discrete feature.
    # Class 0 predominantly gets value 10 but with a small chance for 20,
    # and class 1 predominantly gets value 20 but with a small chance for 10.
    f2_class0 = np.random.choice([10, 20], size=n0, p=[0.9, 0.1])
    f2_class1 = np.random.choice([20, 10], size=n1, p=[0.9, 0.1])
    f2 = np.concatenate([f2_class0, f2_class1])

    # Feature 3: Irrelevant constant feature.
    f3 = np.full((n_samples,), 3.0)

    # Assemble features into one array.
    X = np.column_stack([f0, f1, f2, f3]).astype(np.float32)

    # Shuffle the dataset (so the classes are randomly interleaved)
    idx = np.arange(n_samples)
    np.random.shuffle(idx)
    X = X[idx]
    y = y[idx]
    return X, y

X,y = generate_large_robust_dataset()
model = SKMultiSURF()
model.fit(X, y)
print(model.feature_importances_)
model = SURFSK()
model.fit(X, y)
print(model.feature_importances_)
model = ReliefFSK()
model.fit(X, y)
print(model.feature_importances_)

KeyboardInterrupt: 

In [ ]:
from src.fast_select import MultiSURF, ReliefF, SURF
import numpy as np
def generate_large_robust_dataset(n_samples=10000, seed=59):
    np.random.seed(seed)
    # Create balanced binary classes
    n0 = n_samples // 2
    n1 = n_samples - n0
    y = np.array([0] * n0 + [1] * n1, dtype=np.int32)

    # Feature 0: Highly relevant continuous feature.
    # Class 0 samples come from N(1, 1), class 1 samples come from N(10, 1).
    f0_class0 = np.random.normal(loc=1.0, scale=1.0, size=n0)
    f0_class1 = np.random.normal(loc=10.0, scale=1.0, size=n1)
    f0 = np.concatenate([f0_class0, f0_class1])

    # Feature 1: Irrelevant noise feature.
    # Values drawn from a standard normal distribution regardless of class.
    f1 = np.random.normal(loc=0.0, scale=1.0, size=n_samples)

    # Feature 2: Relevant discrete feature.
    # Class 0 predominantly gets value 10 but with a small chance for 20,
    # and class 1 predominantly gets value 20 but with a small chance for 10.
    f2_class0 = np.random.choice([10, 20], size=n0, p=[0.9, 0.1])
    f2_class1 = np.random.choice([20, 10], size=n1, p=[0.9, 0.1])
    f2 = np.concatenate([f2_class0, f2_class1])

    # Feature 3: Irrelevant constant feature.
    f3 = np.full((n_samples,), 3.0)

    # Assemble features into one array.
    X = np.column_stack([f0, f1, f2, f3]).astype(np.float32)

    # Shuffle the dataset (so the classes are randomly interleaved)
    idx = np.arange(n_samples)
    np.random.shuffle(idx)
    X = X[idx]
    y = y[idx]
    return X, y

X,y = generate_large_robust_dataset()
model = MultiSURF(discrete_limit=4, use_star=True)
model.fit(X, y)
print(model.feature_importances_)
X,y = generate_large_robust_dataset()
model = MultiSURF(discrete_limit=4, backend='cpu', use_star=True)
model.fit(X, y)
print(model.feature_importances_)
model = ReliefF(discrete_limit=4)
model.fit(X, y)
print(model.feature_importances_)
model = ReliefF(discrete_limit=4, backend='cpu')
model.fit(X, y)
print(model.feature_importances_)
model = SURF(discrete_limit=4, use_star=True)
model.fit(X, y)
print(model.feature_importances_)
model = SURF(discrete_limit=4, backend='cpu', use_star=True)
model.fit(X, y)
print(model.feature_importances_)

In [3]:
import numpy as np
from numba import cuda
import psutil
from src.fast_select.MDR import GPUMDRClassifier

# Check if a GPU is available
if not cuda.is_available():
    print("No CUDA-enabled GPU found. Cannot run the MDR example.")
else:
    # --- NEW: Import libraries for memory profiling ---
    import os
    import psutil

    print(f"CUDA is available. Using: {cuda.get_current_device().name.decode('UTF-8')}\n")

    # --- NEW: Helper functions to report memory usage ---
    def bytes_to_mb(b):
        """Converts bytes to megabytes."""
        return round(b / (1024**2), 2)

    def get_gpu_mem_usage():
        """Returns used GPU memory in MB."""
        free, total = cuda.current_context().get_memory_info()
        return bytes_to_mb(total - free)

    # Get the current Python process for CPU memory tracking
    process = psutil.Process(os.getpid())
    # --------------------------------------------------

    # 1. Create Synthetic Data with a Known Interaction
    print("--- Creating Synthetic Data ---")
    n_samples = 1600
    n_features = 200
    np.random.seed(42)

    X = np.random.randint(0, 3, size=(n_samples, n_features), dtype=np.int32)
    y = np.zeros(n_samples, dtype=np.int32)

    interaction_snps = (25, 75)
    print(f"Embedding a pure epistatic interaction between features {interaction_snps}.")
    print("Rule: Case if (feature_25 == 1) AND (feature_75 == 1)")

    risk_mask = (X[:, interaction_snps[0]] == 1) & (X[:, interaction_snps[1]] == 1)
    case_indices = np.where(risk_mask)[0]
    n_cases_to_set = n_samples // 2
    y[case_indices] = 1
    current_cases = np.sum(y)
    if current_cases < n_cases_to_set:
        n_needed = n_cases_to_set - current_cases
        potential_indices = np.where(y == 0)[0]
        promo_indices = np.random.choice(potential_indices, n_needed, replace=False)
        y[promo_indices] = 1

    print(f"Dataset created with {n_samples} samples, {n_features} features.")
    print(f"Class distribution: {np.sum(y==1)} cases, {np.sum(y==0)} controls.\n")


    # 2. Instantiate and Run the GPUMDRClassifier
    mdr = GPUMDRClassifier(k=4, cv=5)

    # --- NEW: Memory reporting before fit ---
    print("--- Memory Usage Before Fit ---")
    mem_before_cpu = process.memory_info().rss
    print(f"CPU Memory Usage: {bytes_to_mb(mem_before_cpu)} MB")
    print(f"GPU Memory Usage: {get_gpu_mem_usage()} MB")
    print("-" * 33)
    # ----------------------------------------

    # Fit the model
    mdr.fit(X, y)

    # --- NEW: Memory reporting after fit ---
    print("\n--- Memory Usage After Fit ---")
    mem_after_cpu = process.memory_info().rss
    print(f"CPU Memory Usage: {bytes_to_mb(mem_after_cpu)} MB")
    print(f"Peak CPU memory during fit: ~{bytes_to_mb(mem_after_cpu - mem_before_cpu)} MB")
    print(f"GPU Memory Usage: {get_gpu_mem_usage()} MB (reflects data currently on GPU)")
    print("-" * 32)

    print("\n--- Model Inspection ---")
    if mdr.best_interaction_ == interaction_snps or mdr.best_interaction_ == tuple(reversed(interaction_snps)):
        print(f"SUCCESS: The classifier correctly identified the embedded interaction: {mdr.best_interaction_}")
    else:
        print(f"INFO: The classifier identified {mdr.best_interaction_} as the strongest signal.")

    print("\n--- Prediction Example ---")
    X_new = np.random.randint(0, 3, size=(10, n_features), dtype=np.int32)
    X_new[3, interaction_snps[0]] = 1
    X_new[3, interaction_snps[1]] = 1

    y_pred = mdr.predict(X_new)
    print(f"Genotypes of new samples for features {mdr.best_interaction_}:\n{X_new[:, mdr.best_interaction_]}")
    print(f"Predicted classes (0=Low-Risk, 1=High-Risk): {y_pred}")

CUDA is available. Using: NVIDIA RTX A2000

--- Creating Synthetic Data ---
Embedding a pure epistatic interaction between features (25, 75).
Rule: Case if (feature_25 == 1) AND (feature_75 == 1)
Dataset created with 1600 samples, 200 features.
Class distribution: 800 cases, 800 controls.

--- Memory Usage Before Fit ---
CPU Memory Usage: 342.28 MB
GPU Memory Usage: 298.38 MB
---------------------------------
Starting 5-fold CV to find best 4-way interaction among 64684950 combinations...
  Fold 1/5: Best model (0, 25, 75, 149), Test BA: 0.6062
  Fold 2/5: Best model (25, 57, 75, 155), Test BA: 0.5656
  Fold 3/5: Best model (25, 75, 78, 115), Test BA: 0.5813
  Fold 4/5: Best model (23, 25, 75, 94), Test BA: 0.5906
  Fold 5/5: Best model (25, 43, 58, 75), Test BA: 0.5219

--- Fit Complete ---
Best Interaction Model Found: (0, 25, 75, 149)
Cross-Validation Consistency (CVC): 1/5
Mean Testing Balanced Accuracy: 0.6062

--- Memory Usage After Fit ---
CPU Memory Usage: 385.96 MB
Peak CPU me

In [2]:
%cd ..

/home/galynch/snap/snapd-desktop-integration/253/Desktop/FastSelect
